# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
# Center-pivot Irrigation Detector.
#
# Finds circles that are 500m in radius.
view_state = pdk.ViewState(longitude=-106.06, latitude=37.71, zoom=12)

# A nice NDVI palette.
palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']

# Just display the image with the palette.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_034034_20170608')
ndvi = image.normalizedDifference(['B5','B4'])

ee_layers.append(EarthEngineLayer(ee_object=ndvi, vis_params={'min':0,'max':1,'palette':palette}))

# Find the difference between convolution with circles and squares.
# This difference, in theory, will be strongest at the center of
# circles in the image. This region is filled with circular farms
# with radii on the order of 500m.
farmSize = 500  # Radius of a farm, in meters.
circleKernel = ee.Kernel.circle(farmSize, 'meters')
squareKernel = ee.Kernel.square(farmSize, 'meters')
circles = ndvi.convolve(circleKernel)
squares = ndvi.convolve(squareKernel)
diff = circles.subtract(squares)

# Scale by 100 and find the best fitting pixel in each neighborhood.
diff = diff.abs().multiply(100).toByte()
max = diff.focal_max(**{'radius': farmSize * 1.8, 'units': 'meters'})
# If a pixel isn't the local max, set it to 0.
local = diff.where(diff.neq(max), 0)
thresh = local.gt(2)

# Here, we highlight the maximum differences as "Kernel Peaks"
# and draw them in red.
peaks = thresh.focal_max(**{'kernel': circleKernel})
ee_layers.append(EarthEngineLayer(ee_object=peaks.updateMask(peaks), vis_params={'palette':'FF3737'}))

# Detect the edges of the features.  Discard the edges with lower intensity.
canny = ee.Algorithms.CannyEdgeDetector(ndvi, 0)
canny = canny.gt(0.3)

# Create a "ring" kernel from two circular kernels.
inner = ee.Kernel.circle(farmSize - 20, 'meters', False, -1)
outer = ee.Kernel.circle(farmSize + 20, 'meters', False, 1)
ring = outer.add(inner, True)

# Highlight the places where the feature edges best match the circle kernel.
centers = canny.convolve(ring).gt(0.5).focal_max({'kernel': circleKernel})
ee_layers.append(EarthEngineLayer(ee_object=centers.updateMask(centers), vis_params={'palette':'4285FF'}))




Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…